In [1]:
import pandas as pd
import numpy as np
import os
import glob
import copy

### Find company with shortest number of days

In [2]:
filenames = glob.glob("new_hist_price/*.csv")

min_length = 100000
min_ind = 0
min_path = ''

for j in range(len(filenames)):

    PATH = filenames[j]

    data = pd.read_csv(PATH)
    df = pd.DataFrame(data, columns= ['v','c','t'])
    data = copy.deepcopy(df.to_numpy())
    
    if np.shape(data)[0] < min_length:
        min_length = np.shape(data)[0]
        min_ind = j
        min_path = PATH
        
print(min_length, min_ind, min_path)

1293 16 new_hist_price\CB_2015-12-30_2021-02-21_minute.csv


### Generate Input X and Input Y For Single Company

In [3]:
# Input X --------------------------------------------------------------
filenames = glob.glob("new_hist_price/*.csv")

PATH = filenames[0]

data = pd.read_csv (PATH)
df = pd.DataFrame(data, columns= ['v','c','t'])
data = df.to_numpy()[:1292,:]

h = np.shape(data)[0]-31

conv_data = np.zeros((32,32,h))

for i in range(h):
    
    vol_min = data[i:i+32, 0].min()
    vol_max = data[i:i+32, 0].max()
    cost_min = data[i:i+32, 1].min()
    cost_max = data[i:i+32, 1].max()

    temp_data = copy.deepcopy(data[i:i+32, :])
    temp_data[:,0] = (temp_data[:,0] - vol_min)/(vol_max - vol_min) * 14
    temp_data[:,1] = (temp_data[:,1] - cost_min)/(cost_max - cost_min) * 14

    temp_data = np.rint(temp_data)
    temp_data[:,0] = (temp_data[:,0] - 14) * -1
    temp_data[:,1] = ((temp_data[:,1] - 14) * -1) + 17
    
    for j in range(32):
        conv_data[int(temp_data[j,0]),j,i] = 1
        conv_data[int(temp_data[j,1]),j,i] = 1
    
conv_data = conv_data.astype(np.int)

with open('inputX_AAL_single.txt', 'w') as outfile:
    for i in range(np.shape(conv_data)[2]):
        cut_2d = conv_data[:,:,i]
        np.savetxt(outfile, cut_2d,fmt='%i')
        if i == (np.shape(conv_data)[2]-1):
            outfile.write('F' + "\n")
            break
        outfile.write('E' + "\n")
        
        
# Input Y --------------------------------------------------------------
PATH = filenames[0]

data = pd.read_csv (PATH)
df = pd.DataFrame(data, columns= ['v','c','t'])
data = df.to_numpy()[:1293,:]

h = np.shape(data)[0]-32

y_vals = []
twenty_count = 0

for i in range(h):
    cur_cost = data[i+31,1]
    next_cost = data[i+32,1]
    
    if next_cost < cur_cost*0.8:
        next_cost = cur_cost*0.8
        twenty_count += 1
    elif next_cost > cur_cost*1.2:
        next_cost = cur_cost*1.2
        twenty_count += 1
        
    y_vals.append(100 * (next_cost-cur_cost)/cur_cost)

y_vals = np.array(y_vals)

with open('inputY_AAL_single.txt', 'w') as outfile:
    for i in range(np.shape(y_vals)[0]):
        np.savetxt(outfile, [y_vals[i]],fmt='%1.4f')  

### Generate Input X and Input Y For Training Companies

In [4]:
# Input X --------------------------------------------------------------
filenames = glob.glob("new_hist_price/*.csv")

with open('inputX_50_companies.txt', 'w') as outfile:

    for k in range(len(filenames)-14):

        PATH = filenames[k]
        print(PATH)

        data = pd.read_csv (PATH)
        df = pd.DataFrame(data, columns= ['v','c','t'])
        data = df.to_numpy()[:1292,:]

        h = np.shape(data)[0]-31

        conv_data = np.zeros((32,32,h))

        for i in range(h):

            vol_min = data[i:i+32, 0].min()
            vol_max = data[i:i+32, 0].max()
            cost_min = data[i:i+32, 1].min()
            cost_max = data[i:i+32, 1].max()

            temp_data = copy.deepcopy(data[i:i+32, :])
            temp_data[:,0] = (temp_data[:,0] - vol_min)/(vol_max - vol_min) * 14
            temp_data[:,1] = (temp_data[:,1] - cost_min)/(cost_max - cost_min) * 14

            temp_data = np.rint(temp_data)
            temp_data[:,0] = (temp_data[:,0] - 14) * -1
            temp_data[:,1] = ((temp_data[:,1] - 14) * -1) + 17

            for j in range(32):
                conv_data[int(temp_data[j,0]),j,i] = 1
                conv_data[int(temp_data[j,1]),j,i] = 1

        conv_data = conv_data.astype(np.int)
        
        for i in range(np.shape(conv_data)[2]):
            cut_2d = conv_data[:,:,i]
            np.savetxt(outfile, cut_2d,fmt='%i')
            if i == (np.shape(conv_data)[2]-1):
                outfile.write('F' + "\n")
                break
            outfile.write('E' + "\n")
            
            
# Input Y --------------------------------------------------------------
with open('inputY_50_companies.txt', 'w') as outfile:

    for k in range(len(filenames)-14):

        PATH = filenames[k]

        data = pd.read_csv (PATH)
        df = pd.DataFrame(data, columns= ['v','c','t'])
        data = df.to_numpy()[:1293,:]

        h = np.shape(data)[0]-32

        y_vals = []
        twenty_count = 0

        for i in range(h):
            cur_cost = data[i+31,1]
            next_cost = data[i+32,1]

            if next_cost < cur_cost*0.8:
                next_cost = cur_cost*0.8
                twenty_count += 1
            elif next_cost > cur_cost*1.2:
                next_cost = cur_cost*1.2
                twenty_count += 1

            y_vals.append(100 * (next_cost-cur_cost)/cur_cost)

        y_vals = np.array(y_vals)
        # print(np.shape(y_vals), twenty_count)

        for i in range(np.shape(y_vals)[0]):
            np.savetxt(outfile, [y_vals[i]],fmt='%1.4f')  

new_hist_price\AAL_2015-12-30_2021-02-21_minute.csv
new_hist_price\AAPL_2015-12-30_2021-02-21_minute.csv
new_hist_price\ABBV_2015-12-30_2021-02-21_minute.csv
new_hist_price\ABT_2015-12-30_2021-02-21_minute.csv
new_hist_price\AIG_2015-12-30_2021-02-21_minute.csv
new_hist_price\ALB_2015-12-30_2021-02-21_minute.csv
new_hist_price\AMGN_2015-12-30_2021-02-21_minute.csv
new_hist_price\AMZN_2015-12-30_2021-02-21_minute.csv
new_hist_price\AVGO_2015-12-30_2021-02-21_minute.csv
new_hist_price\AXP_2015-12-30_2021-02-21_minute.csv
new_hist_price\BAC_2015-12-30_2021-02-21_minute.csv
new_hist_price\BA_2015-12-30_2021-02-21_minute.csv
new_hist_price\BIIB_2015-12-30_2021-02-21_minute.csv
new_hist_price\BLK_2015-12-30_2021-02-21_minute.csv
new_hist_price\CAT_2015-12-30_2021-02-21_minute.csv
new_hist_price\CBOE_2015-12-30_2021-02-21_minute.csv
new_hist_price\CB_2015-12-30_2021-02-21_minute.csv
new_hist_price\CMCSA_2015-12-30_2021-02-21_minute.csv
new_hist_price\CME_2015-12-30_2021-02-21_minute.csv
new_h

### Generate Input X and Input Y For Testing Companies

In [5]:
# Input X --------------------------------------------------------------
filenames = glob.glob("new_hist_price/*.csv")

with open('inputX_14_companies.txt', 'w') as outfile:

    for k in range(14):

        PATH = filenames[k+50]
        print(PATH)

        data = pd.read_csv (PATH)
        df = pd.DataFrame(data, columns= ['v','c','t'])
        data = df.to_numpy()[:1292,:]

        h = np.shape(data)[0]-31

        conv_data = np.zeros((32,32,h))

        for i in range(h):

            vol_min = data[i:i+32, 0].min()
            vol_max = data[i:i+32, 0].max()
            cost_min = data[i:i+32, 1].min()
            cost_max = data[i:i+32, 1].max()

            temp_data = copy.deepcopy(data[i:i+32, :])
            temp_data[:,0] = (temp_data[:,0] - vol_min)/(vol_max - vol_min) * 14
            temp_data[:,1] = (temp_data[:,1] - cost_min)/(cost_max - cost_min) * 14

            temp_data = np.rint(temp_data)
            temp_data[:,0] = (temp_data[:,0] - 14) * -1
            temp_data[:,1] = ((temp_data[:,1] - 14) * -1) + 17

            for j in range(32):
                conv_data[int(temp_data[j,0]),j,i] = 1
                conv_data[int(temp_data[j,1]),j,i] = 1

        conv_data = conv_data.astype(np.int)
        
        for i in range(np.shape(conv_data)[2]):
            cut_2d = conv_data[:,:,i]
            np.savetxt(outfile, cut_2d,fmt='%i')
            if i == (np.shape(conv_data)[2]-1):
                outfile.write('F' + "\n")
                break
            outfile.write('E' + "\n")
                        

# Input Y --------------------------------------------------------------
with open('inputY_14_companies.txt', 'w') as outfile:

    for k in range(14):

        PATH = filenames[k+50]

        data = pd.read_csv (PATH)
        df = pd.DataFrame(data, columns= ['v','c','t'])
        data = df.to_numpy()[:1293,:]
        
        h = np.shape(data)[0]-32

        y_vals = []
        twenty_count = 0

        for i in range(h):
            cur_cost = data[i+31,1]
            next_cost = data[i+32,1]

            if next_cost < cur_cost*0.8:
                next_cost = cur_cost*0.8
                twenty_count += 1
            elif next_cost > cur_cost*1.2:
                next_cost = cur_cost*1.2
                twenty_count += 1

            y_vals.append(100 * (next_cost-cur_cost)/cur_cost)

        y_vals = np.array(y_vals)
        # print(np.shape(y_vals), twenty_count)

        for i in range(np.shape(y_vals)[0]):
            np.savetxt(outfile, [y_vals[i]],fmt='%1.4f')  

new_hist_price\REGN_2015-12-30_2021-02-21_minute.csv
new_hist_price\SLB_2015-12-30_2021-02-21_minute.csv
new_hist_price\SO_2015-12-30_2021-02-21_minute.csv
new_hist_price\TGT_2015-12-30_2021-02-21_minute.csv
new_hist_price\TMO_2015-12-30_2021-02-21_minute.csv
new_hist_price\TMUS_2015-12-30_2021-02-21_minute.csv
new_hist_price\TSLA_2015-12-30_2021-02-21_minute.csv
new_hist_price\TSN_2015-12-30_2021-02-21_minute.csv
new_hist_price\TWTR_2015-12-30_2021-02-21_minute.csv
new_hist_price\T_2015-12-30_2021-02-21_minute.csv
new_hist_price\UAL_2015-12-30_2021-02-21_minute.csv
new_hist_price\VLO_2015-12-30_2021-02-21_minute.csv
new_hist_price\WFC_2015-12-30_2021-02-21_minute.csv
new_hist_price\XRX_2015-12-30_2021-02-21_minute.csv
